# imports and paths

In [ ]:
import os
import csv
import time
import math
import yaml
import subprocess
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, STATUS_FAIL
from threading import Lock
from collections import deque


path_home =         "/home/fabian"
path_catkin_ws =    path_home + "/catkin_ws"
path_ros_package =  path_catkin_ws + "/src/drive_ros_config/modules/drive_ros_imu_odo_odometry"
path_config_file =  path_ros_package + "/config/phoenix_cc2017.yaml"
path_trial_runner = path_ros_package + "/scripts/runTrial.sh"
path_bag_file =     "/data/Bags/Drive/drive_2017-06-16-14-31-35.bag"
path_results =      "/data/KalmanTuningLogs/"

# update covariances
load default .yaml file, change kalman covariances and save as temporary .yaml file

In [ ]:
def update_config(x, y, a, v, theta, omega, config_file_out):
    
    # load config
    config_file_in = path_config_file
    config_in = open(config_file_in, 'r')
    config = yaml.load(config_in)
    
    # update config
    config['kalman_cov']['sys_var_x'] = x;
    config['kalman_cov']['sys_var_y'] = y;
    config['kalman_cov']['sys_var_a'] = a;
    config['kalman_cov']['sys_var_v'] = v;
    config['kalman_cov']['sys_var_theta'] = theta;
    config['kalman_cov']['sys_var_omega'] = omega;
    
    # save config
    config_out = open(config_file_out, 'w')
    yaml.dump(config, config_out)

# trial number
when we run in parallel, we have to keep track on the correct trial number and use locks

In [ ]:
ct = 0
lock = Lock()

def increase_trial():
    global ct
    lock.acquire()
    ct = ct + 1
    local_ct = ct
    lock.release()
    return local_ct

# evaluate funciton
this function evaluates the output and returns a single loss function

In [ ]:
def evaluate(results_csv):
    # open csv file
    with open(results_csv, 'r') as f:
        try:
            lastrow = deque(csv.reader(f), 1)[0]
        except IndexError:  # empty file
            lastrow = None
            
        x = float(lastrow[2]) # x position at the end
        y = float(lastrow[3]) # y position at the end
        return math.sqrt(x*x + y*y) # distance from start to the end (assume we drive a circle)

# objective function
for each trial this function is called and starts the trial runner script

In [ ]:
def objective(params):
    # overwrite parameters and use default
    #params = {'x': 0.001, 'y': 0.001, 'a': 0.0024, 'v': 0.0004, 'theta': 0.008, 'omega': 0.03}

    # get trial number
    trial = increase_trial()

    # check if trial number already exists
    while os.path.exists(path_results + str(trial)):
        trial = increase_trial()

    # create directory for this trial
    os.makedirs(path_results + str(trial))

    # config file path
    yaml_config = path_results + str(trial) + "/config.yaml"

    # create config file for this trial
    update_config(params['x'], params['y'], params['a'], params['v'],
                  params['theta'], params['omega'], yaml_config)

    # start trail runner
    p = subprocess.run([path_trial_runner, "--trail", str(ct), 
                                           "--logdir", path_results + str(trial),
                                           "--bag", path_bag_file,
                                           "--config", yaml_config,
                                           "--catkin_ws", path_catkin_ws], shell=False)
    
    # evaluate return code of trail runner
    if 0 == p.returncode:
        status = STATUS_OK
    else:
        status = STATUS_FAIL
    
    # return trail results
    return {
        'loss': evaluate(path_results + str(trial) + "/odom.csv"),
        'status': status,
        'eval_time': time.time(),
        'log_path': path_results + str(trial)
        }

# search for hyperparameter
here we create the space and search for the best hyperparameters

In [ ]:
space = {
    'x': hp.uniform('x', 0, 1),
    'y': hp.uniform('y', 0, 1),
    'a': hp.uniform('a', 0, 1),
    'v': hp.uniform('v', 0, 1),
    'theta': hp.uniform('theta', 0, 1),
    'omega': hp.uniform('omega', 0, 1)
}

algo = tpe.suggest
max_evals = 3
trials = Trials()

best = fmin(objective, space, algo, max_evals, trials)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(trials.losses())
plt.ylabel('some numbers')
plt.show()